In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from imblearn.metrics import sensitivity_specificity_support
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from IPython.core.display import display, HTML
from sklearn import preprocessing

<h2>Funções</h2>
<h15><hr></h15>
<h4>- Foram usadas duas funções que retornarão o erro médio absoluto e os valores previstos</h4>
<h4>- Cada função retornará o relativo a função de regressão que foi chamada</h4>
<h4>- Para cada função de regressão pode ser alterada uma métrica diferente, visando obter os melhores resultados</h4>
<h4>- Logistic Regression pode ser alterado o parâmetro C, e Decision Tree altera a quantidade máxima de nós folha</h4>

In [3]:
def get_mae_DT(max_leaf_nodes, train_X, val_x, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=7)
    model.fit(train_X, train_y)
    pred = model.predict(val_X)
    mae = mean_absolute_error(val_y, pred)
    return mae, pred
    


def get_mae_LR(C, train_X, val_X, train_y, val_y):
    model = LogisticRegression(random_state=3, multi_class='auto', max_iter=1000, C=C, solver='liblinear')
    model.fit(train_X, train_y)
    pred = model.predict(val_X)
    mae = mean_absolute_error(val_y, pred)
    return mae, pred

<h4>- Nessa parte foram alterados os valores não numéricos da base de dados para valores numéricos</h4>
<h4>- Houve, outrossim, o tratamento de um erro na leitura do csv que, ao tratar labels = 'NA'  considerava-as como celulás sem valor algum</h4>
<h4>- Isso foi feito utilizendo o fillna(0.0)</h4>

In [4]:
house_file = 'train2.csv'
house_data = pd.read_csv(house_file)

news = house_data.fillna(0.0)

    
new_filtered_house_data = news.replace({'Lvl': 2, 'Bnk': 1.5, 'HLS': 1,'Low': 0.0 , 'AllPub': 4,
'Inside': 1, 'Corner': 1.25, 'CulDSac': 1.5, 'FR2': 2, 'FR3': 2.5, 'Grvl': 1, 'Pave': 2,'ELO': 1, 
'Reg': 2, 'IR1': 1.5, 'IR2': 1, 'IR3': 0.0, 'Gtl': 3, 'Mod': 2, 'Sev': 1, '1Fam': 1, '2fmCon': 1.5,
'Duplex': 2, 'TwnhsE': 2.5, 'TwnhsI': 3, 'Twnhs': 3.5, '1Story': 1, '1.5Fin': 1.5, '1.5Unf': 1.25,
'2.5Fin': 2.5, '2.5Unf': 2.25, 'SFoyer': 3, 'SLvl': 2.75, 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1,
'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'PAN': 0.0, 'Y': 1, 'N': 0.0, 'Typ': 8,
'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 5, 'Sev': 3, 'Sal': 2, 'Fin': 3, 'RFn': 2, 'P' : 0.0,
'GdWo': 3, 'MnWw': 2, 'BrkTil': 5, 'CBlock': 4, 'PConc': 3, 'Slab': 2, 'Stone': 1, 'Wood':0,
'Mn': 1, 'Av': 2, 'SBrkr':5, 'FuseA':4, 'FuseF':3, 'FuseP':2, 'Mix': 1, 'Elev': 5, 'Gar2':4,
'TenC':6, 'WD': 10, 'CWD':9, 'VWD':8, 'New':7, 'COD': 6, 'Con':5, 'ConLw': 4, 'ConLI':3, 'ConLD':2, 
'NoSeWa':2.5,'Plywood':7, 'PreCast':6, 'Stucco':5,'Somerst':5,'MeadowV':15,'Blueste':24,'VinylSd':4,
'Abnorml':5, 'AdjLand':4, 'Alloca':3, 'Family':2,'Partial':1, 'Artery': 9, 'Feedr': 8, 'Norm':7, 
'RRNe':4, 'RRAe':3, 'PosA':10, 'A': 9, 'C':8, 'FV':7, 'I':6, 'RH':5, 'RL':4, 'RP':3, 'RM':2, 
'BrDale':23, 'BrkSide':22, 'ClearCr':21, 'CollgCr':20, 'Crawfor':19, 'Edwards': 18, 'Gilbert':17,
'Mitchel':14, 'NAmes':13, 'NoRidge':12, 'NPkVill':11, 'NWames':10, 'OldTown':9, 'SWISU':8,
'StoneBr':4, 'Timber':3, 'Veenker':2, 'Floor':6, 'GasA':5, 'GasW':4, 'Grav':3, 'OthW':2, 'Wall':1, 
'Gable':5, 'WdShngl':1, 'MetalSd':9,'SawyerW':6,'MnPrv': 4,'RRAn': 5,'Shed':1,'No': 0.5,'Wd Sdng':3,
'ImStucc':10,'C (all)':6.5,'IDOTRR':16,'Oth':2.5,'Normal': 6,'2Story': 2,'NoSewr':3,'PosN': 10, 
'Gambrel':4, 'CarPort':3, 'Detchd':2, 'NWAmes':2.5, 'NridgHt':1.5, 'CmentBd':4.5, 'Brk Cmn':3.5,
'Min1':7,'Other':8,'WdShake':2,'Blmngtn': 25,'RRNn':6,'Flat':6,'GdPrv':5,'Othr':2.5,'Sawyer':7,
'Hip':3, 'Mansard':2, 'ClyTile':8, 'CompShg':7, 'Membran':6, 'Metal':5, 'Roll':4, 'Tar&Grv':3,
'Attchd':6, 'None':0, 'BrkCmn': 15, '2Types':7,'Basment':4.5,'BuiltIn':4,'Wd Shng':2.5, 'WdShing':2, 
'AsbShng':17, 'AsphShn':16, 'BrkComm':15, 'BrkFace':14, 'CBlock':13, 'CemntBd':12, 'HdBoard':11,})

media_sale_price = new_filtered_house_data['SalePrice'].mean(axis=0, skipna=True)
print(media_sale_price)

180921.19589041095


<h4>Nesta célula ocorre a atribuição dos valores para previsão (features), no qual foram retirados da base completa apenas as colunas de ID e de SalePrice que é o que se deseja prever (target), também havendo uma normalização dos dados</h4>

In [5]:
bf_X = np.array(new_filtered_house_data.drop(['SalePrice', 'Id'], axis=1))
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(bf_X)
y = new_filtered_house_data.SalePrice

<h1>Logistic Regression</h1>
<h15><hr></h15>
<h4>- Um modelo para variáveis dependentes ou binomialmente distribuídas, foi utilizado um solver lblinear que abarca bem datasets não tão grandes</h4>
<h4>- O valor de C foi sucessivamente alterado, sendo C o inverso da regularização, quanto menor o C maior a regularização e menor a chance de overfitting</h4>
<h4>- Pois a regularização penaliza parâmetros muito grandes</h4>
<h4>- Alterando esse valor foi obtido o menor erro absoluto de 34345 e um r2 score de 0.5, para um C valendo 10</h4>
<h4>- Para achar os melhores foi usado o coeficiente de determinação r2, que demonstra um score da função de regressão, onde o máximo valor é 1, e pode ser negativo caso a previsão desvie muito</h4>

In [6]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1, test_size=0.20)
C = [0.001,0.01,0.1,1,10,100]

for i in C:
    new_mae_lr, pred = get_mae_LR(i, train_X, val_X, train_y, val_y)
    print("Para o parametro C (inverso da regularização) de: {0:.3f}\nO erro médio absoluto de: {1:.3f}\n".format(i, new_mae_lr))
    print('r2 score: {}'.format(r2_score(val_y, pred)))
    print('-'*50)

Para o parametro C (inverso da regularização) de: 0.001
O erro médio absoluto de: 58082.106

r2 score: -0.1794293328295289
--------------------------------------------------
Para o parametro C (inverso da regularização) de: 0.010
O erro médio absoluto de: 53236.216

r2 score: 0.07027128745206979
--------------------------------------------------
Para o parametro C (inverso da regularização) de: 0.100
O erro médio absoluto de: 43279.949

r2 score: 0.26683422200822693
--------------------------------------------------
Para o parametro C (inverso da regularização) de: 1.000
O erro médio absoluto de: 39215.438

r2 score: 0.43664183102839693
--------------------------------------------------
Para o parametro C (inverso da regularização) de: 10.000
O erro médio absoluto de: 34345.438

r2 score: 0.5017262866960572
--------------------------------------------------
Para o parametro C (inverso da regularização) de: 100.000
O erro médio absoluto de: 35267.455

r2 score: 0.46877228522177206
-----

<h1>Decision Tree Regression</h1>
<h15><hr></h15>
<h4>- Algoritmo baseado em árvores de decisão que como dito na questão anterior são muito bons para problemas com tabelas e variáveis mais claras, diferente de imagens por exemplo, na qual o ideal é uma RNA</h4>
<h4>- Alterando o parâmetro de profundidade máxima da árvore de decisão criada, aumentando muito esse valor pode levar a overfitting pois a base pode aprender de dados que são ruídos</h4>
<h4>- Assim, ao observar os valores obtidos, percebe-se que o melhor foi dado por uma profundidade máxima de 50, com um erro médio absoluto de: 21922, e um r2 score de 0.86</h4>

In [7]:
# Testando com outros parametros 
for max_leaf_nodes in [5, 10, 25, 50, 100, 500, 1000, 10000]:
    new_mae, pred = get_mae_DT(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Máximo de nós folha: {0}\nErro médio: {1:.4f}\n" .format(max_leaf_nodes, new_mae))
    print('r2 score: {}'.format(r2_score(val_y, pred)))
    print('-'*50)

Máximo de nós folha: 5
Erro médio: 30982.1258

r2 score: 0.7118391379121103
--------------------------------------------------
Máximo de nós folha: 10
Erro médio: 27406.2150

r2 score: 0.813199458646888
--------------------------------------------------
Máximo de nós folha: 25
Erro médio: 24202.2407

r2 score: 0.8489887122138565
--------------------------------------------------
Máximo de nós folha: 50
Erro médio: 21922.5534

r2 score: 0.8674798485584855
--------------------------------------------------
Máximo de nós folha: 100
Erro médio: 22462.1671

r2 score: 0.8508177390992901
--------------------------------------------------
Máximo de nós folha: 500
Erro médio: 23589.9639

r2 score: 0.831405039041653
--------------------------------------------------
Máximo de nós folha: 1000
Erro médio: 24168.1404

r2 score: 0.8261697912411099
--------------------------------------------------
Máximo de nós folha: 10000
Erro médio: 24165.6507

r2 score: 0.8261492509546979
-----------------------